In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from gcn import GCN

import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
from dgl import AddSelfLoop
import argparse

import wandb
import argparse
from monitor import Monitor

In [2]:
def evaluate(g, features, labels, mask, model):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)


In [3]:
def train(g, features, labels, masks, model):
    monitor = Monitor()

    # define train/val samples, loss function and optimizer
    train_mask = masks[0]
    val_mask = masks[1]
    loss_fcn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-4)

    # training loop
    for epoch in range(1000):
        model.train()
        logits = model(g, features)
        m_1 = monitor.get_sys_info()            
            
        loss = loss_fcn(logits[train_mask], labels[train_mask])
        m_2 = monitor.get_sys_info()
        
        optimizer.zero_grad()
        loss.backward()        
        m_3 = monitor.get_sys_info()
        
#         keys_1 = list(map(lambda ls: ls+"_forward", m_1.keys()))
#         keys_2 = list(map(lambda ls: ls+"_loss", m_1.keys()))
#         keys_3 = list(map(lambda ls: ls+"_backward", m_1.keys()))
#         wandb.log(dict(zip(keys_1,m_1.values())))
#         wandb.log(dict(zip(keys_2,m_1.values())))
#         wandb.log(dict(zip(keys_3,m_1.values())))
        wandb.log(m_3)
        
        

        
        
        optimizer.step()
        acc = evaluate(g, features, labels, val_mask, model)
        print("Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f} "
              . format(epoch, loss.item(), acc))


In [4]:



run = wandb.init(project = 'GNN-Conv-Networks', entity='kubework')


hidden_layers = [64,32,16,8]

data = CoraGraphDataset(transform= AddSelfLoop())
g = data[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
g = g.int().to(device)
features = g.ndata['feat']
labels = g.ndata['label']
masks = g.ndata['train_mask'], g.ndata['val_mask'], g.ndata['test_mask']

# normalization
degs = g.in_degrees().float()
norm = torch.pow(degs, -0.5).to(device)
norm[torch.isinf(norm)] = 0
g.ndata['norm'] = norm.unsqueeze(1)

# create GCN model    
in_size = features.shape[1]
out_size = data.num_classes
model = GCN(in_size, hidden_layers, out_size).to(device)

# model training
print('Training...')
train(g, features, labels, masks, model)

# test the model
print('Testing...')
acc = evaluate(g, features, labels, masks[2], model)
print("Test accuracy {:.4f}".format(acc))

wandb: Currently logged in as: alessandroferrenti (kubework). Use `wandb login --relogin` to force relogin


  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Training...
Epoch 00000 | Loss 1.9458 | Accuracy 0.0580 
Epoch 00001 | Loss 1.9433 | Accuracy 0.1360 
Epoch 00002 | Loss 1.9425 | Accuracy 0.1240 


/home/ale/anaconda3/envs/wandb/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 00003 | Loss 1.9400 | Accuracy 0.3600 
Epoch 00004 | Loss 1.9402 | Accuracy 0.3340 
Epoch 00005 | Loss 1.9357 | Accuracy 0.3320 
Epoch 00006 | Loss 1.9202 | Accuracy 0.4220 
Epoch 00007 | Loss 1.9045 | Accuracy 0.4880 
Epoch 00008 | Loss 1.9036 | Accuracy 0.4860 
Epoch 00009 | Loss 1.8679 | Accuracy 0.4080 
Epoch 00010 | Loss 1.8637 | Accuracy 0.3500 
Epoch 00011 | Loss 1.8332 | Accuracy 0.3540 
Epoch 00012 | Loss 1.8193 | Accuracy 0.3980 
Epoch 00013 | Loss 1.7628 | Accuracy 0.4380 
Epoch 00014 | Loss 1.7316 | Accuracy 0.4760 
Epoch 00015 | Loss 1.7019 | Accuracy 0.4260 
Epoch 00016 | Loss 1.6765 | Accuracy 0.4300 
Epoch 00017 | Loss 1.5844 | Accuracy 0.4380 
Epoch 00018 | Loss 1.5168 | Accuracy 0.4340 
Epoch 00019 | Loss 1.5571 | Accuracy 0.3600 
Epoch 00020 | Loss 1.5644 | Accuracy 0.3420 
Epoch 00021 | Loss 1.4434 | Accuracy 0.4180 
Epoch 00022 | Loss 1.3796 | Accuracy 0.4200 
Epoch 00023 | Loss 1.3810 | Accuracy 0.4440 
Epoch 00024 | Loss 1.3932 | Accuracy 0.4280 
Epoch 0002

Epoch 00186 | Loss 0.3219 | Accuracy 0.7020 
Epoch 00187 | Loss 0.2859 | Accuracy 0.7100 
Epoch 00188 | Loss 0.1889 | Accuracy 0.7020 
Epoch 00189 | Loss 0.3506 | Accuracy 0.6900 
Epoch 00190 | Loss 0.2059 | Accuracy 0.6840 
Epoch 00191 | Loss 0.2189 | Accuracy 0.6820 
Epoch 00192 | Loss 0.2583 | Accuracy 0.6880 
Epoch 00193 | Loss 0.1989 | Accuracy 0.6960 
Epoch 00194 | Loss 0.2793 | Accuracy 0.7160 
Epoch 00195 | Loss 0.2100 | Accuracy 0.6940 
Epoch 00196 | Loss 0.1631 | Accuracy 0.6860 
Epoch 00197 | Loss 0.1741 | Accuracy 0.7080 
Epoch 00198 | Loss 0.2142 | Accuracy 0.6900 
Epoch 00199 | Loss 0.2670 | Accuracy 0.6760 
Epoch 00200 | Loss 0.2300 | Accuracy 0.6880 
Epoch 00201 | Loss 0.2414 | Accuracy 0.7000 
Epoch 00202 | Loss 0.2355 | Accuracy 0.7100 
Epoch 00203 | Loss 0.2870 | Accuracy 0.7100 
Epoch 00204 | Loss 0.2898 | Accuracy 0.7160 
Epoch 00205 | Loss 0.1884 | Accuracy 0.7120 
Epoch 00206 | Loss 0.2700 | Accuracy 0.7220 
Epoch 00207 | Loss 0.3213 | Accuracy 0.7240 
Epoch 0020

Epoch 00372 | Loss 0.1552 | Accuracy 0.7220 
Epoch 00373 | Loss 0.1192 | Accuracy 0.7220 
Epoch 00374 | Loss 0.1255 | Accuracy 0.7000 
Epoch 00375 | Loss 0.1417 | Accuracy 0.7120 
Epoch 00376 | Loss 0.0974 | Accuracy 0.7220 
Epoch 00377 | Loss 0.0992 | Accuracy 0.7200 
Epoch 00378 | Loss 0.1407 | Accuracy 0.7000 
Epoch 00379 | Loss 0.0913 | Accuracy 0.6660 
Epoch 00380 | Loss 0.1425 | Accuracy 0.7200 
Epoch 00381 | Loss 0.1722 | Accuracy 0.7400 
Epoch 00382 | Loss 0.0993 | Accuracy 0.7280 
Epoch 00383 | Loss 0.1330 | Accuracy 0.7220 
Epoch 00384 | Loss 0.1360 | Accuracy 0.7180 
Epoch 00385 | Loss 0.1475 | Accuracy 0.7320 
Epoch 00386 | Loss 0.2158 | Accuracy 0.7140 
Epoch 00387 | Loss 0.1687 | Accuracy 0.7060 
Epoch 00388 | Loss 0.1167 | Accuracy 0.6940 
Epoch 00389 | Loss 0.1070 | Accuracy 0.6880 
Epoch 00390 | Loss 0.1635 | Accuracy 0.7160 
Epoch 00391 | Loss 0.1220 | Accuracy 0.7080 
Epoch 00392 | Loss 0.2100 | Accuracy 0.7140 
Epoch 00393 | Loss 0.0966 | Accuracy 0.7240 
Epoch 0039

Epoch 00557 | Loss 0.0831 | Accuracy 0.7120 
Epoch 00558 | Loss 0.1323 | Accuracy 0.7180 
Epoch 00559 | Loss 0.3107 | Accuracy 0.7200 
Epoch 00560 | Loss 0.1521 | Accuracy 0.7120 
Epoch 00561 | Loss 0.0901 | Accuracy 0.6920 
Epoch 00562 | Loss 0.2057 | Accuracy 0.6840 
Epoch 00563 | Loss 0.1514 | Accuracy 0.7160 
Epoch 00564 | Loss 0.0990 | Accuracy 0.7160 
Epoch 00565 | Loss 0.1753 | Accuracy 0.6960 
Epoch 00566 | Loss 0.1390 | Accuracy 0.6880 
Epoch 00567 | Loss 0.1382 | Accuracy 0.6980 
Epoch 00568 | Loss 0.1437 | Accuracy 0.7040 
Epoch 00569 | Loss 0.1461 | Accuracy 0.7060 
Epoch 00570 | Loss 0.2181 | Accuracy 0.7140 
Epoch 00571 | Loss 0.1390 | Accuracy 0.7220 
Epoch 00572 | Loss 0.1144 | Accuracy 0.7300 
Epoch 00573 | Loss 0.2020 | Accuracy 0.7280 
Epoch 00574 | Loss 0.1438 | Accuracy 0.7240 
Epoch 00575 | Loss 0.1516 | Accuracy 0.7220 
Epoch 00576 | Loss 0.1118 | Accuracy 0.7140 
Epoch 00577 | Loss 0.1818 | Accuracy 0.7200 
Epoch 00578 | Loss 0.1770 | Accuracy 0.7100 
Epoch 0057

Epoch 00742 | Loss 0.1108 | Accuracy 0.7100 
Epoch 00743 | Loss 0.0797 | Accuracy 0.7160 
Epoch 00744 | Loss 0.1542 | Accuracy 0.7200 
Epoch 00745 | Loss 0.1118 | Accuracy 0.7220 
Epoch 00746 | Loss 0.1598 | Accuracy 0.7300 
Epoch 00747 | Loss 0.1213 | Accuracy 0.7260 
Epoch 00748 | Loss 0.1446 | Accuracy 0.7300 
Epoch 00749 | Loss 0.1496 | Accuracy 0.7320 
Epoch 00750 | Loss 0.1167 | Accuracy 0.7380 
Epoch 00751 | Loss 0.1041 | Accuracy 0.7400 
Epoch 00752 | Loss 0.1181 | Accuracy 0.7360 
Epoch 00753 | Loss 0.0828 | Accuracy 0.7080 
Epoch 00754 | Loss 0.1289 | Accuracy 0.7240 
Epoch 00755 | Loss 0.1372 | Accuracy 0.7380 
Epoch 00756 | Loss 0.1314 | Accuracy 0.7260 
Epoch 00757 | Loss 0.1100 | Accuracy 0.6980 
Epoch 00758 | Loss 0.1468 | Accuracy 0.7220 
Epoch 00759 | Loss 0.1445 | Accuracy 0.7340 
Epoch 00760 | Loss 0.0837 | Accuracy 0.7320 
Epoch 00761 | Loss 0.1167 | Accuracy 0.7160 
Epoch 00762 | Loss 0.1373 | Accuracy 0.7360 
Epoch 00763 | Loss 0.1441 | Accuracy 0.7280 
Epoch 0076

Epoch 00928 | Loss 0.0598 | Accuracy 0.7380 
Epoch 00929 | Loss 0.0719 | Accuracy 0.7440 
Epoch 00930 | Loss 0.0828 | Accuracy 0.7460 
Epoch 00931 | Loss 0.1552 | Accuracy 0.7400 
Epoch 00932 | Loss 0.1205 | Accuracy 0.7280 
Epoch 00933 | Loss 0.0989 | Accuracy 0.7260 
Epoch 00934 | Loss 0.0841 | Accuracy 0.7140 
Epoch 00935 | Loss 0.0984 | Accuracy 0.6800 
Epoch 00936 | Loss 0.2008 | Accuracy 0.6840 
Epoch 00937 | Loss 0.1885 | Accuracy 0.7240 
Epoch 00938 | Loss 0.0838 | Accuracy 0.7440 
Epoch 00939 | Loss 0.1450 | Accuracy 0.7120 
Epoch 00940 | Loss 0.0867 | Accuracy 0.7320 
Epoch 00941 | Loss 0.1236 | Accuracy 0.7480 
Epoch 00942 | Loss 0.1103 | Accuracy 0.7320 
Epoch 00943 | Loss 0.1396 | Accuracy 0.7260 
Epoch 00944 | Loss 0.1877 | Accuracy 0.7060 
Epoch 00945 | Loss 0.1733 | Accuracy 0.6840 
Epoch 00946 | Loss 0.0697 | Accuracy 0.6780 
Epoch 00947 | Loss 0.0960 | Accuracy 0.7040 
Epoch 00948 | Loss 0.1820 | Accuracy 0.7260 
Epoch 00949 | Loss 0.1609 | Accuracy 0.7300 
Epoch 0095

In [5]:
monitor = Monitor()
m_1 = monitor.get_sys_info()
m_2 = monitor.get_sys_info()
m_3 = monitor.get_sys_info()

keys = list(map(lambda ls: ls+"_forward", m_1.keys()))
dict(zip(keys,m_1.values()))

{'cpu/logical_core_count_forward': 8,
 'cpu/interrupts/ctx_switches_count_forward': 50,
 'cpu/interrupts/interrupts_count_forward': 21,
 'cpu/interrupts/soft_interrupts_count_forward': 20,
 'cpu/load/avg_sys_load_one_min_percent_forward': 84.25,
 'cpu/load/avg_sys_load_five_min_percent_forward': 36.25,
 'cpu/load/avg_sys_load_fifteen_min_percent_forward': 30.125,
 'memory/total_memory_sys_mb_forward': 7677.671875,
 'memory/available_memory_sys_mb_forward': 1528.953125,
 'memory/used_memory_sys_mb_forward': 5053.94140625,
 'memory/used_memory_sys_percent_forward': 80.1,
 'process/voluntary_proc_ctx_switches_forward': 12521,
 'process/involuntary_proc_ctx_switches_forward': 7079,
 'process/memory/resident_set_size_proc_mb_forward': 373.45703125,
 'process/memory/virtual_memory_size_proc_mb_forward': 2329.83984375,
 'process/memory/shared_memory_proc_mb_forward': 104.4609375,
 'process/memory/text_resident_set_proc_mb_forward': 2.33984375,
 'process/memory/data_resident_set_proc_mb_forwar